In [1]:
import findspark

In [2]:
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [5]:
from pyspark.ml.regression import LinearRegression

In [7]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [8]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [11]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [12]:
from pyspark.ml.linalg import Vectors

In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'],
                           outputCol='features')

In [15]:
output = assembler.transform(data)

In [16]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [17]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [18]:
final_data = output.select('features', 'Yearly Amount Spent')

In [19]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [20]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [21]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                348|
|   mean|  500.0139945266261|
| stddev|  79.99440109658862|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [22]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                152|
|   mean|  497.7115068028253|
| stddev|  77.97420361497053|
|    min|  275.9184206503857|
|    max|  689.7876041747194|
+-------+-------------------+



In [23]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [24]:
lr_model = lr.fit(train_data)

In [25]:
test_results = lr_model.evaluate(test_data)

In [26]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  8.470083814303734|
|  -12.4037530545462|
| -5.153152657582666|
|-14.107339869183022|
| -22.56450062953138|
|-0.6233497369062206|
| -9.022925865758339|
| -5.002129086243315|
|-10.153838307797002|
| 2.5559506687462203|
| -18.83355452045248|
|-15.631999865207263|
|  5.595862557905264|
| -5.987728941650005|
|-12.379744074633663|
| -3.630183494106859|
| -10.13636084167814|
|   7.75815287418169|
|-2.9617047518948993|
| -9.671948296566768|
+-------------------+
only showing top 20 rows



In [27]:
test_results.rootMeanSquaredError

10.826590146642115

In [28]:
test_results.r2

0.9805934588380738

In [29]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [30]:
unlabeled_data = test_data.select('features')

In [31]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.3931845423455...|
|[30.8794843441274...|
|[31.0662181616375...|
|[31.1239743499119...|
|[31.3895854806643...|
|[31.4474464941278...|
|[31.5171218025062...|
|[31.5261978982398...|
|[31.5316044825729...|
|[31.5702008293202...|
|[31.5741380228732...|
|[31.6548096756927...|
|[31.7656188210424...|
|[31.8093003166791...|
|[31.8124825597242...|
|[31.8279790554652...|
|[31.8512531286083...|
|[31.8627411090001...|
|[31.8854062999117...|
+--------------------+
only showing top 20 rows



In [32]:
predictions = lr_model.transform(unlabeled_data)

In [33]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...|400.17026725832375|
|[30.3931845423455...| 332.3326228577398|
|[30.8794843441274...|495.35975264243734|
|[31.0662181616375...| 463.0406330768574|
|[31.1239743499119...|509.51155446929715|
|[31.3895854806643...| 410.6929607968891|
|[31.4474464941278...|427.62566796098235|
|[31.5171218025062...|  280.920549736629|
|[31.5261978982398...| 419.2483645001348|
|[31.5316044825729...|433.95965506061634|
|[31.5702008293202...| 564.7790466618574|
|[31.5741380228732...| 560.0412720257941|
|[31.6548096756927...|469.66756116964325|
|[31.7656188210424...|502.54181057725714|
|[31.8093003166791...| 549.1516434374748|
|[31.8124825597242...| 396.4405284779041|
|[31.8279790554652...|450.13910838861966|
|[31.8512531286083...| 465.2340937926167|
|[31.8627411090001...| 559.2598459259416|
|[31.8854062999117...| 399.7752212690423|
+--------------------+------------